# 주요 시설
- 장소 이름
- 주소 
- 편의시설 
- 편의시설의 이용가능과 이용불편(허가 기준치 이상/미달 설치) 
- 이용가능과 이용불편들의 상세설명 

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import sys
from time import sleep

In [2]:
headers = {
    "user-agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36"
}

parms = {
    "pagenum" : 1,
    "Depth" : None
}

In [3]:
url = "http://disability.seoul.go.kr/amenity/sightSeeingConven/sightSeeingConven.jsp"

In [4]:
def getUrl(url, parameter=None, num_retries=2, select=None):
    
    html = requests.get(url, params=parameter, headers=headers)
    
    if 500 <= html.status_code  < 600 and num_retries > 0:
        print(html.status_code, html.reason)
        return getUrl(url, params, num_retries=1)
        
    dom = BeautifulSoup(html.text, "lxml")
    links = dom.select(select)
    
    return links

In [5]:
select_css = { 'place': 'ul.travel-list2 > li > dl > dt > a ',
               'address' : 'ul.travel-list2 > li > dl > dd > p',
               'phone' : '.tel',
               'facility': 'ul.travel-list2 > li > dl > dd > ul.facil' }

In [6]:
place_Depth_code = {"movie/concerthall": 4111, "shopping/mart": 4211,
                    "publicparking": 4311,  "publictoilet":4411 , 
                    "hospital" : 4511,"publicinstitution" : 4611,
                    "bank" : 4711, "education/Childcare" : 4811, 
                    "beauty/hair" : 4911, "other" :4121}

# 1. 영화관/공연장 2. 쇼핑/마트 3. 공영주차장 4. 공영화장실 5. 의료 6. 공공기관 
# 7. 은행 8. 교육/보육 9.이.미용실 10. 기타

In [7]:
movie = []
shopping = []
publicparking = []
publictoilet = []
hospital = []
publicinstitution = []
bank = []
education = []
beauty = []
other = []

In [8]:
# 1. 영화관/공연장 2. 쇼핑/마트 3. 공영주차장 4. 공영화장실 5. 의료 6. 공공기관 
# 7. 은행 8. 교육/보육 9.이.미용실 10. 기타
# type 구조 [{ }] 리스안에 딕셔너리 구조 
final_dict = {
'movie/concerthall' : None ,
'shopping/mart' : None ,
'publicparking' : None ,
'publictoilet' : None ,
'hospital' : None ,
'publicinstitution' : None,
'bank' : None,
'education/Childcare' : None,
'beauty/hair' : None,
'other' : None,
}
info_dict = {}
result = [] 

In [ ]:
for place in place_Depth_code:
    place_code = place_Depth_code[place]
    parms['Depth'] = place_code
    result = []  
    while True:
        for select in select_css:
            get_info = getUrl(url, parameter = parms, select = select_css[select])
            # 장소, 장소_사이트, 주소, 전화번호, 편의시설
            if 'place' is select:
                place_name = [info.text for info in get_info] # 장소
                place_href = [info['href'] for info in get_info] # 장소_사이트
            if 'address' is select:
                address_pre = [re.sub(u'\xa0 \r\n\t*',"",info.text) for info in get_info] # 주소
                address =[re.sub(u'\n|주소 :  ',"",info) for info in address_pre]
#             if 'phone' is select:
#                 phone = [info.text for info in get_info] # 전화번호
#             if 'facility' is select:
#                 facility_pre_select = [info for info in get_info] # 편의시설
                
        if get_info: 
            for info_num in range(0, len(place_name)):
#                 facility_select =  facility_pre_select[info_num].select('li > img')
#                 facility_select = [row_facility['alt'] for row_facility in facility_select]
#                 print(facility_select)
                info_dict = {'place_name' : place_name[info_num],
                             'place_href' : 'http://disability.seoul.go.kr'+place_href[info_num],
                             'place_address' : address[info_num],
#                              'place_facility' : facility_select
                             }  #     #
                result.append(info_dict)
            print(info_dict)
            parms['pagenum'] = parms['pagenum'] + 1
            # pagenum ->for next page
        else:
            parms['pagenum'] = 1
            break
    final_dict[place] = result

In [10]:
number_len =0 
for place in place_Depth_code:
    number_len += len(final_dict[place])
print(number_len)

4856


In [11]:
facility_select =  "dl.accordion" # 필요한 부분만 자르기

In [12]:
facility_available_name_select = "dt > a" # 시설물 설치된 이름
facility_is_available_select = "dt > span" # 시설물 이용가능 or 이용 불편 
facility_info_select = "ul.bulListDot > li" # 시설물 상세 정보 

In [13]:
final_dicts = {}
info_result = [] 
for place in place_Depth_code:

    for i, info in enumerate(final_dict[place]):
        info_result = [] 
        get_info = getUrl(info['place_href'], select = facility_select)
        facility_available_name= [info.text for info in get_info[0].select(facility_available_name_select)]
        facility_is_available = [info.text for info in get_info[0].select(facility_is_available_select)]
        facility_info = [info for info in get_info[0].select(facility_info_select)]
        facility_info = [info.__str__() for info in facility_info]
        facility_info = [ re.sub('^<li>|-|(<br/>)?</li>$' , '' , info).split('<br/>') for info in facility_info]
        
        for info_num in range(0, len(facility_available_name)):
            info_dict  = {'facility_available_name' : facility_available_name[info_num],
                     'facility_is_available' : facility_is_available[info_num],
                     'facility_info' : facility_info[info_num]
                     }
            info_result.append(info_dict)
        a = str(i)
        final_dict[place][i]['facility_detail_info'] = info_result
        info_result = None 
# print( final_dict[place][i]['facility_detail_info'])
#         print( final_dict[place][i]['facility_detail_info'])
#         print(final_dict[place][i]['facility_detail_info'])

In [14]:
import json
import urllib.request
import urllib.request
import datetime
import time
import json

In [15]:
def get_request_url(url):
    
    client_id="R63uNWNQiaTp1EeZl7bW"
    client_secret = "1Im1JohssC"
    
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)
    try: 
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print ("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None
    
def getGeoData(address):
    
    base = "https://openapi.naver.com/v1/map/geocode"
    node = ""
    parameters = "?query=%s" % urllib.parse.quote(address)
    url = base + node + parameters
    
    retData = get_request_url(url)
    
    if (retData == None):
        return None
    else:
        return json.loads(retData)

In [ ]:
for place in place_Depth_code:
    info_result = [] 
    print(place)
    
    for info_num,info in enumerate(final_dict[place]):
        print(info['place_name'])
        a = str(info['place_address'])
        jsonResult = getGeoData(a)
        
        try:
            for item in jsonResult['result']['items']:

                    final_dict[place][info_num]['lat'] =  str(item['point']['y'])
                    final_dict[place][info_num]['lon'] =  str(item['point']['x'])
        except Exception as e:
            print("예외!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            final_dict[place][info_num]['lat'] = "NaN"
            final_dict[place][info_num]['lon']= "NaN"    
            print(e)

In [18]:
with open('main_place.json', 'w', encoding="utf-8") as fp:
    json.dump(final_dict, fp, ensure_ascii=False, indent="\t")

In [48]:
import json

In [47]:
with open('main_place.json', 'r', encoding='utf-8') as fp:
    final_dict = json.load(fp)

In [49]:
type(final_dict)

dict

# 1. 주요시설 이름/ 주요시설 주소/ 경도/ 위도 / 카테고리

In [50]:
import pandas as pd

In [109]:
datas = DataFrame()

for info in final_dict.keys():
    data = pd.DataFrame.from_dict(final_dict[info])
    data = pd.DataFrame(data, columns=['place_name' , 'place_address', 'lat', 'lon'])
    data['category'] = info 
    data['lat'] = data['lat'].astype(float)
    data['lon'] = data['lon'].astype(float)
    datas = datas.append(data)


In [110]:
a = [i for i in range(0,4856)]
a[4855]

4855

In [111]:
datas['place_id'] =a

In [112]:
datas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4856 entries, 0 to 908
Data columns (total 6 columns):
place_name       4856 non-null object
place_address    4856 non-null object
lat              4739 non-null float64
lon              4739 non-null float64
category         4856 non-null object
place_id         4856 non-null int64
dtypes: float64(2), int64(1), object(3)
memory usage: 265.6+ KB


In [113]:
datas.head()

,place_name,place_address,lat,lon,category,place_id
0,A아트홀(구.신연아트홀),서울특별시 종로구 동숭동 1-37,37.583269,127.002443,movie/concerthall,0
1,BBC씨어터,서울특별시 강남구 신사동 568-2,37.523930,127.025143,movie/concerthall,1
2,BREAK OUT 전용극장,서울특별시 종로구 관철동 33-1,37.568835,126.988123,movie/concerthall,2
3,BYRDS(버즈),서울특별시 마포구 와우산로 51,37.549100,126.922860,movie/concerthall,3
4,Between the Bars,서울특별시 마포구 어울마당로 114,37.554833,126.923213,movie/concerthall,4


In [115]:
datas.reset_index(inplace=True)

In [116]:
datas.describe()

,index,lat,lon,place_id
count,4856.000000,4739.000000,4739.000000,4856.000000
mean,663.729819,37.552996,126.986674,2427.500000
std,586.480708,0.053958,0.088598,1401.950784
min,0.000000,37.437812,126.797912,0.000000
25%,185.000000,37.507579,126.912847,1213.750000
50%,496.000000,37.549768,126.999287,2427.500000
75%,937.250000,37.589668,127.056813,3641.250000
max,2151.000000,37.694797,127.178341,4855.000000


In [95]:
datas.set_index('place_id', inplace=True)

In [260]:
datas.to_csv("main_place.csv",encoding = "utf-8")

In [16]:
pd.unique(datas['place_name'])

array(['A아트홀(구.신연아트홀)', 'BBC씨어터', 'BREAK OUT 전용극장', ..., '흑석체육센터',
       '흥원사가양동법당', '흥황교회'], dtype=object)

# 2. 주요시설 이름/주요시설물 설치/ (이용가능 /이용불편)

In [97]:
data_facility = DataFrame()
i = 0 
for test in final_dict.keys():
    for category in final_dict[test]:
        
        data_pre_facility = pd.DataFrame.from_dict(category['facility_detail_info'])
        data_pre_facility = pd.DataFrame(data_pre_facility, columns=['facility_available_name', 'facility_is_available'])
        data_pre_facility['place_name'] = category['place_name']
        data_pre_facility['place_id'] = i
          
        data_facility = data_facility.append(data_pre_facility)
        i += 1
data_facility['facility_available_name'] += "_"+data_facility['place_name']

In [99]:
data_facility.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16617 entries, 0 to 2
Data columns (total 4 columns):
facility_available_name    16617 non-null object
facility_is_available      16617 non-null object
place_name                 16617 non-null object
place_id                   16617 non-null int64
dtypes: int64(1), object(3)
memory usage: 649.1+ KB


In [100]:
data_facility.head()

,facility_available_name,facility_is_available,place_name,place_id
0,주출입구 접근로_A아트홀(구.신연아트홀),이용불편,A아트홀(구.신연아트홀),0
1,주출입구 높이차이 제거_A아트홀(구.신연아트홀),이용가능,A아트홀(구.신연아트홀),0
0,주출입구 접근로_BBC씨어터,이용가능,BBC씨어터,1
1,장애인 전용 주차구역_BBC씨어터,이용가능,BBC씨어터,1
2,주출입구 높이차이 제거_BBC씨어터,이용가능,BBC씨어터,1


In [101]:
data_facility.reset_index(inplace=True)

In [102]:
data_facility.head()

,index,facility_available_name,facility_is_available,place_name,place_id
0,0,주출입구 접근로_A아트홀(구.신연아트홀),이용불편,A아트홀(구.신연아트홀),0
1,1,주출입구 높이차이 제거_A아트홀(구.신연아트홀),이용가능,A아트홀(구.신연아트홀),0
2,0,주출입구 접근로_BBC씨어터,이용가능,BBC씨어터,1
3,1,장애인 전용 주차구역_BBC씨어터,이용가능,BBC씨어터,1
4,2,주출입구 높이차이 제거_BBC씨어터,이용가능,BBC씨어터,1


In [103]:
x = data_facility.apply(lambda x : str(x['place_id'])+"-"+str(x['index']),axis=1)

In [104]:
data_facility['facility_id'] =x

In [174]:
data_facility.drop(columns=['index'], inplace=True)

In [255]:
data_facility
# facility_id = place_id + facility_availabe_name

,facility_available_name,facility_is_available,place_name,place_id,facility_id
0,주출입구 접근로_A아트홀(구.신연아트홀),이용불편,A아트홀(구.신연아트홀),0,0-0
1,주출입구 높이차이 제거_A아트홀(구.신연아트홀),이용가능,A아트홀(구.신연아트홀),0,0-1
2,주출입구 접근로_BBC씨어터,이용가능,BBC씨어터,1,1-0
3,장애인 전용 주차구역_BBC씨어터,이용가능,BBC씨어터,1,1-1
4,주출입구 높이차이 제거_BBC씨어터,이용가능,BBC씨어터,1,1-2
5,장애인용 승강기_BBC씨어터,이용가능,BBC씨어터,1,1-3
6,장애인용 관람석_BBC씨어터,이용가능,BBC씨어터,1,1-4
7,주출입구 접근로_BREAK OUT 전용극장,이용가능,BREAK OUT 전용극장,2,2-0
8,장애인 전용 주차구역_BREAK OUT 전용극장,이용불편,BREAK OUT 전용극장,2,2-1
9,장애인용 승강기_BREAK OUT 전용극장,이용가능,BREAK OUT 전용극장,2,2-2


In [258]:
pd.DataFrame.to_csv(data_facility, "main_place_facility_available.csv", encoding = "utf-8")

# 3. 주요시설 상세정보 / 주요시설물 설치

In [177]:
import re

In [247]:
data_facility_detail = DataFrame()
for test in final_dict.keys():
    for category in final_dict[test]:

        for facility in category['facility_detail_info']:
            a = [info for info in facility['facility_info']]

            data_pre_facility = pd.DataFrame.from_dict(a)
           
            data_pre_facility['facility_available_name'] = facility['facility_available_name']+"_"+category['place_name']
            get_facility_id = data_facility[data_pre_facility['facility_available_name'][0] == data_facility['facility_available_name']]['facility_id']
            get_facility_id = re.findall(r'([0-9]+-[0-9]+)',str(get_facility_id))
            get_facility_id = [get_facility_id[0] for i in data_pre_facility['facility_available_name']]
            data_pre_facility['facility_id'] = get_facility_id
            data_facility_detail = data_facility_detail.append(data_pre_facility)
            

In [248]:
data_facility_detail = data_facility_detail.rename(index=str, columns={0: "facility_detail_info"})

In [249]:
data_facility_detail = data_facility_detail.reset_index(drop=True)

In [250]:
data_facility_detail

,facility_detail_info,facility_available_name,facility_id
0,평탄함,주출입구 접근로_A아트홀(구.신연아트홀),0-0
1,단차없음,주출입구 접근로_A아트홀(구.신연아트홀),0-0
2,기울기 1/18~1/12이하,주출입구 접근로_A아트홀(구.신연아트홀),0-0
3,보/차도 구분,주출입구 접근로_A아트홀(구.신연아트홀),0-0
4,상세정보 없음,주출입구 높이차이 제거_A아트홀(구.신연아트홀),0-1
5,평탄함,주출입구 접근로_BBC씨어터,1-0
6,단차없음,주출입구 접근로_BBC씨어터,1-0
7,유효폭 1.2m 이상,주출입구 접근로_BBC씨어터,1-0
8,기울기 1/18이하,주출입구 접근로_BBC씨어터,1-0
9,보/차도 구분,주출입구 접근로_BBC씨어터,1-0


In [259]:
pd.DataFrame.to_csv(data_facility_detail, "main_place_facility_detail_info.csv",encoding = "utf-8")